In [1]:
import requests
from edgar_functions import *
from lxml import etree

ticker = "AMD"

In [2]:
accession_num_series = get_10K_accessionNumbers_for_ticker(ticker)

In [3]:
filing = get_specific_filing_using_accessionNumber(ticker, accession_num_series[0])

/var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_22089/2558583301.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  filing = get_specific_filing_using_accessionNumber(ticker, accession_num_series[0])


In [4]:
filing = pd.read_html(filing)

/var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_22089/1571899766.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  filing = pd.read_html(filing)


In [5]:
def filter_accession_number_index_page(df, column_name, search_term):
    fitered_df = df[df[column_name].str.contains(search_term, case=False, na=False)]
    filtered_list = fitered_df[column_name].tolist()
    return filtered_list

In [6]:
def rename_statement(statement, label_dictionary):
    statement.index = statement.index.map(lambda x: label_dictionary.get(x, x))
    return statement

In [7]:
def rename_with_all_statement_labels(statement, ticker):
    accession_num_series = get_10K_accessionNumbers_for_ticker(ticker)
    for accession_num in accession_num_series:
        label_dict = get_label_dictionary(ticker, accession_num)
        if label_dict == None:
            break
        statement = rename_statement(statement, label_dict)
        print("Renamed with accession number: " + accession_num)
    return statement

In [8]:
def get_schema_document_links(ticker, accession_number):
    cik = get_cik_matching_ticker(ticker)
    headers = {"User-Agent": "russ@sunriseanalysis.com"}
    base_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number}"
    requested_content = requests.get(base_url, headers=headers).content.decode("utf-8")
    filing = pd.read_html(requested_content)
    defenition_link = filter_accession_number_index_page(filing[0], "Name", "_def.xml")
    presentation_link = filter_accession_number_index_page(
        filing[0], "Name", "_pre.xml"
    )
    label_link = filter_accession_number_index_page(filing[0], "Name", "_lab.xml")
    defenition_link = f"{base_url}/{defenition_link[0]}"
    presentation_link = f"{base_url}/{presentation_link[0]}"
    label_link = f"{base_url}/{label_link[0]}"
    return defenition_link, presentation_link, label_link

In [9]:
def get_label_dictionary(ticker, accession_number):
    cik = get_cik_matching_ticker(ticker)
    headers = {"User-Agent": "russ@sunriseanalysis.com"}
    base_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number}"
    requested_content = requests.get(base_url, headers=headers).content.decode("utf-8")
    filing = pd.read_html(requested_content)
    try:
        label_link = filter_accession_number_index_page(filing[0], "Name", "_lab.xml")
    except:
        return None
    label_link = f"{base_url}/{label_link[0]}"
    label_content = requests.get(label_link, headers=headers).content
    tree_lab = etree.fromstring(label_content)
    namespaces = tree_lab.nsmap
    label_dictionary = {}
    for label in tree_lab.xpath("//link:label", namespaces=namespaces):
        gaap_fact = label.get("{http://www.w3.org/1999/xlink}label", None).split(
            "_", 1
        )[-1]
        human_readable_label = label.text
        label_dictionary[gaap_fact] = human_readable_label
    return label_dictionary

In [10]:
def_link, pre_link, lab_link = get_schema_document_links(
    ticker, accession_num_series[0]
)
print(def_link)
print(lab_link)

/var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_22089/1867963748.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ticker, accession_num_series[0]


https://www.sec.gov/Archives/edgar/data/0000002488/000000248823000047/amd-20221231_def.xml
https://www.sec.gov/Archives/edgar/data/0000002488/000000248823000047/amd-20221231_lab.xml


/var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_22089/2724415213.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  filing = pd.read_html(requested_content)


In [11]:
label_dict = get_label_dictionary(ticker, accession_num_series[0])

/var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_22089/3147092235.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label_dict = get_label_dictionary(ticker, accession_num_series[0])
/var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_22089/2076979527.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  filing = pd.read_html(requested_content)


In [12]:
balance_sheet = pd.read_csv("quarterly&annual_results/wsm/balance_sheet_annual.csv")

In [19]:
balance_sheet = form_statement_for_all_available_years(ticker, "balance_sheet")

In [20]:
balance_sheet

,2022-12-31,2021-12-25,2020-12-26,2019-12-28,2018-12-29,2017-12-30,2016-12-31,2015-12-26,2014-12-27,2013-12-28
us-gaap_AssetsCurrentAbstract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_CashAndCashEquivalentsAtCarryingValue,4835.0,2535.0,1595.0,1466.0,1078.0,1185.0,1264.0,785.0,805.0,869.0
us-gaap_ShortTermInvestments,1020.0,1073.0,695.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_AccountsReceivableNetCurrent,4126.0,2706.0,2066.0,1859.0,1235.0,454.0,311.0,533.0,818.0,832.0
us-gaap_InventoryNet,3771.0,1955.0,1399.0,982.0,845.0,694.0,751.0,678.0,685.0,884.0
us-gaap_RelatedPartyTransactionDueFromToRelatedPartyCurrent,2.0,2.0,10.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_PrepaidExpenseAndOtherAssetsCurrent,1265.0,312.0,378.0,233.0,270.0,NaN,NaN,NaN,80.0,71.0
us-gaap_AssetsCurrent,15019.0,8583.0,6143.0,4597.0,3540.0,2634.0,2530.0,2320.0,2736.0,2884.0
us-gaap_PropertyPlantAndEquipmentNet,1513.0,702.0,641.0,500.0,348.0,261.0,164.0,188.0,302.0,346.0
us-gaap_OperatingLeaseRightOfUseAsset,460.0,367.0,208.0,205.0,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
balance_sheet = balance_sheet.set_index("Unnamed: 0")

In [21]:
balance_sheet = rename_with_all_statement_labels(balance_sheet, ticker)

/var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_22089/2076979527.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  filing = pd.read_html(requested_content)


Renamed with accession number: 000000248823000047


/var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_22089/2076979527.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  filing = pd.read_html(requested_content)


Renamed with accession number: 000000248822000016


/var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_22089/2076979527.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  filing = pd.read_html(requested_content)


...

/var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_22089/2076979527.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  filing = pd.read_html(requested_content)


Renamed with accession number: 000000248817000043


/var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_22089/2076979527.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  filing = pd.read_html(requested_content)


Renamed with accession number: 000000248816000111
Renamed with accession number: 000119312515054362


/var/folders/8q/0wn9p2n54hb1x_gb6ckxsdxr0000gn/T/ipykernel_22089/2076979527.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  filing = pd.read_html(requested_content)


In [23]:
balance_sheet_single_rename = rename_statement(balance_sheet, label_dict)

In [24]:
balance_sheet_single_rename

,2022-12-31,2021-12-25,2020-12-26,2019-12-28,2018-12-29,2017-12-30,2016-12-31,2015-12-26,2014-12-27,2013-12-28
"Assets, Current [Abstract]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Cash and Cash Equivalents, at Carrying Value",4835.0,2535.0,1595.0,1466.0,1078.0,1185.0,1264.0,785.0,805.0,869.0
Short-Term Investments,1020.0,1073.0,695.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN
"Accounts Receivable, after Allowance for Credit Loss, Current",4126.0,2706.0,2066.0,1859.0,1235.0,454.0,311.0,533.0,818.0,832.0
"Inventory, Net",3771.0,1955.0,1399.0,982.0,845.0,694.0,751.0,678.0,685.0,884.0
"Related Party Transaction, Due from (to) Related Party, Current",2.0,2.0,10.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN
"Prepaid Expense and Other Assets, Current",1265.0,312.0,378.0,233.0,270.0,NaN,NaN,NaN,80.0,71.0
"Assets, Current",15019.0,8583.0,6143.0,4597.0,3540.0,2634.0,2530.0,2320.0,2736.0,2884.0
"Property, Plant and Equipment, Net",1513.0,702.0,641.0,500.0,348.0,261.0,164.0,188.0,302.0,346.0
"Operating Lease, Right-of-Use Asset",460.0,367.0,208.0,205.0,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
label_content = requests.get(lab_link, headers=headers).content.decode("utf-8")

In [18]:
BeautifulSoup(label_content, "lxml")

/Users/russell/miniconda3/envs/Edgar/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<?xml version="1.0" encoding="UTF-8"?><!--XBRL Document Created with Wdesk from Workiva--><!--Copyright 2023 Workiva--><!--r:df3599cc-6ee4-478c-b435-a187f18d6929,g:cb1dc209-bc7f-4425-8304-91091a35488e--><html><body><link:linkbase xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemalocation="http://www.xbrl.org/2003/linkbase http://www.xbrl.org/2003/xbrl-linkbase-2003-12-31.xsd">
<link:roleref roleuri="http://www.xbrl.org/2009/role/negatedPeriodStartLabel" xlink:href="http://www.xbrl.org/lrr/role/negated-2009-12-16.xsd#negatedPeriodStartLabel" xlink:type="simple"></link:roleref>
<link:roleref roleuri="http://www.xbrl.org/2009/role/netLabel" xlink:href="http://www.xbrl.org/lrr/role/net-2009-12-16.xsd#netLabel" xlink:type="simple"></link:roleref>
<link:roleref roleuri="http://www.xbrl.org/2009/role/negatedNetLabel" xlink:href="http://www.xbrl.org/lrr/role/negated-2009-12-16.xsd#negatedNet